In [63]:
try:
    import requests
    import pandas as pd
    import time as time
    import database as db
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys 
except Exception as e:
    print(e)

In [39]:
data = db.DataBase()

In [74]:
ano = "2018"

navegador = webdriver.Chrome()
navegador.get("http://buscatextual.cnpq.br/buscatextual/busca.do")

#Tags de pesquisa
navegador.find_element_by_xpath('//*[@id="buscarDemais"]').send_keys(Keys.SPACE)
navegador.find_element_by_xpath('//*[@id="buscaAssunto"]').send_keys(Keys.SPACE)

#Caixa de texto
buscar = navegador.find_element_by_xpath('//*[@id="textoBusca"]')
buscar.send_keys(ano)
buscar.send_keys(Keys.ENTER)

time.sleep(2)

passadas = []

janelaAtual = navegador.current_window_handle
passadas.append(navegador.window_handles)

#Variaveis de controle da paginação
pagina = 1
pag = 2

while(True):
    #Troca de pagina
    try:
        if pagina!=1 and pag<=11 and pagina<=11:
            navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
            pag = pag+1 if pag<11 else 3

        if pagina>11 and pag>2:
            navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
            pag = pag+1 if pag<13 else 3
    except Exception as e:
        print(e)
        break
        
    time.sleep(2)
    
    
    for i in range(1,11):
        #Selecionar curriculo
        navegador.find_element_by_xpath(f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a').click()

        time.sleep(5)

        #Abrir curriculo
        navegador.find_element_by_xpath('//*[@id="idbtnabrircurriculo"]').click()

        time.sleep(4)

        for janela in navegador.window_handles:
            if janela not in passadas:
                navegador.switch_to.window(janela)

        #Pegar dados 
        site = BeautifulSoup(navegador.page_source, 'html.parser')

        artigos = []

        for artigo in site.findAll('div', attrs={'class':['artigo-completo']}):
            if "2021" in artigo.text:
                artigo = artigo.text[10:-4:]
                text = ""
                maior = 0
                for s in artigo.split("."):
                    if len(s) > maior:
                        text = s
                        maior = len(s)
                artigos.append(text)

        nome = site.findAll('h2', attrs={'class':['nome']})[0].text

        #Inserir no banco de dados
        if artigos != []:
            data.insertData(nome, artigos, ano)

        navegador.close()

        time.sleep(1)

        navegador.switch_to.window(janelaAtual)

        #Fechar curriculo
        navegador.find_element_by_xpath('//*[@id="idbtnfechar"]').click()
    
    data.connection.commit()
    pagina += 1

navegador.quit()

IndexError: list index out of range

In [75]:
data.selectData(2018)

,primeiro_nome,nome_completo,titulo,ano
0,Valdino,Valdino Silva dos Santos Junior,"NETO, JOAQUIM CIALDINE PORTELA2021NETO, JOAQUI...",2018
1,Flávia,Flávia Cruz Moraes Sobral,MORTALIDADE POR ACIDENTES DE TRÂNSITO EM IDOS...,2018
2,Diego,Diego Leonardo Morassutti Desiderio,A DESMISTIFICAÇÃO DA EDUCAÇÃO SEXUAL E A SUA ...,2018
3,Diego,Diego Leonardo Morassutti Desiderio,A DESMISTIFICAÇÃO DA EDUCAÇÃO SEXUAL E A SUA ...,2018
4,Eduardo,Eduardo Ramiro Pasiani,"FERRI, LUANA FERREIRA2021FERRI, LUANA FERREIRA...",2018
5,Eduardo,Eduardo Ramiro Pasiani,"FERRI, LUANA FERREIRA2021FERRI, LUANA FERREIRA...",2018
6,Brenda,Brenda Pereira Teles,Perfil epidemiológico de pacientes Chagásicos...,2018
7,Tânia,Tânia Oliveira dos Santos,Perfil epidemiológico do vírus da dengue em v...,2018
